In [42]:
START_DATE = "2024-08-23"
END_DATE = "2024-08-23"

In [43]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True

In [44]:
import pandas as pd
import re


def add_mesage_to_df(messages, date, time, id, message, type_="message", subtype=None):
    messages.append(
        {
            "Date": date,
            "Time": time,
            "Id": id,
            "Message": message,
            "Type": type_,
            "Action_type": subtype,
        }
    )
    return messages


def manage_action(messages, current_date, current_time, id, rest_of_line):
    if "entrou usando o link de convite deste grupo" in rest_of_line:
        action_type = "entry"
        id = rest_of_line.split("entrou usando o link de convite deste grupo")[
            0
        ].strip()
    elif "saiu" in rest_of_line:
        action_type = "exit"
        id = rest_of_line.split("saiu")[0].strip()
    elif "mudou a descrição do grupo" in rest_of_line:
        action_type = "description_change"
        id = rest_of_line.split("mudou a descrição do grupo")[0].strip()
    elif "fixou uma mensagem" in rest_of_line:
        action_type = "message_pin"
        id = rest_of_line.split("fixou uma mensagem")[0].strip()
    elif "mudou as configurações" in rest_of_line:
        action_type = "group_settings_change"
        id = rest_of_line.split("mudou as configurações do grupo")[0].strip()
    elif "(arquivo anexado)" in rest_of_line:
        action_type = "file_attach"
        id = rest_of_line.split("(arquivo anexado)")[0].strip()
    elif "criou o grupo" in rest_of_line:
        action_type = "group_create"
        id = rest_of_line.split("criou o grupo")[0].strip()
    elif "foi adicionado(a)" in rest_of_line:
        action_type = "add"
        id = rest_of_line.split("foi adicionado(a)")[0].strip()
    else:
        print("Error line:", rest_of_line)
        assert False

    add_mesage_to_df(
        messages, current_date, current_time, id, None, "action", subtype=action_type
    )


def parse_whatsapp_history(file_path):
    # Lista para armazenar as mensagens extraídas
    messages = []

    # Ler o arquivo de histórico
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    # Regex para identificar a data, hora e número do remetente
    date_and_time_pattern = re.compile(r"(\d{2}/\d{2}/\d{4}) (\d{2}:\d{2}) - (.*)")

    # Variáveis para acumular mensagens
    current_date = None
    current_time = None
    current_id = None
    current_message = None

    for line in lines:
        # Verifica se a linha corresponde ao padrão de uma nova mensagem
        match = date_and_time_pattern.match(line)
        if match:
            if current_message:
                messages = add_mesage_to_df(
                    messages, current_date, current_time, current_id, current_message
                )
            current_message = ""
            current_date, current_time, rest_of_line = match.groups()
            # if "entrou usando o link de convite deste grupo" in rest_of_line:
            #     id = rest_of_line.split("entrou usando o link de convite deste grupo")[
            #         0
            #     ].strip()
            #     add_mesage_to_df(messages, current_date, current_time, id, "", "entry")
            if "‎" in rest_of_line:
                manage_action(
                    messages, current_date, current_time, current_id, rest_of_line
                )
            else:
                if ":" in rest_of_line:
                    current_id, message = rest_of_line.split(":", 1)
                    current_message = message.strip()
        else:
            current_message += "\n" + line.strip()

    messages = add_mesage_to_df(
        messages, current_date, current_time, current_id, current_message, "message"
    )

    # Criar DataFrame com as mensagens
    df = pd.DataFrame(messages)

    # Cria linha de ordem
    df["Order"] = range(1, len(df) + 1)
    
    # parse columns date and time into a datetime column
    df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str))

    return df


# Exemplo de uso
file_path = "../conversa.txt"
history = parse_whatsapp_history(file_path)
history[history["Type"] != "message"].head()

/tmp/ipykernel_244459/1335014072.py:109: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str))


,Date,Time,Id,Message,Type,Action_type,Order,DateTime
0,19/08/2024,19:53,‎~ Felipe Soares,None,action,group_create,1,2024-08-19 19:53:00
4,19/08/2024,20:32,‎+55 61 9217-6339,None,action,entry,5,2024-08-19 20:32:00
6,19/08/2024,20:33,‎+55 11 98910-2316,None,action,entry,7,2024-08-19 20:33:00
30,19/08/2024,20:39,‎+55 11 98760-9956,None,action,entry,31,2024-08-19 20:39:00
65,19/08/2024,20:44,‎+55 13 97406-4831,None,action,entry,66,2024-08-19 20:44:00


In [45]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2626 entries, 0 to 2625
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         2626 non-null   object        
 1   Time         2626 non-null   object        
 2   Id           2626 non-null   object        
 3   Message      2540 non-null   object        
 4   Type         2626 non-null   object        
 5   Action_type  86 non-null     object        
 6   Order        2626 non-null   int64         
 7   DateTime     2626 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 164.3+ KB


In [46]:
print("Min DateTime:", history["DateTime"].min())
print("Max DateTime:", history["DateTime"].max())

Min DateTime: 2024-08-19 19:53:00
Max DateTime: 2024-08-24 01:26:00


In [49]:
from datetime import datetime, timedelta

# Convert strings to datetime objects
start_datetime = datetime.strptime(START_DATE, "%Y-%m-%d")
end_datetime = datetime.strptime(END_DATE, "%Y-%m-%d") + timedelta(days=1)

messages_df = history[
    (history["Type"] == "message")
    & (history["DateTime"] >= start_datetime)
    & (history["DateTime"] < end_datetime)
]
print(len(messages_df))
messages_df

397


,Date,Time,Id,Message,Type,Action_type,Order,DateTime
2224,23/08/2024,04:50,Jordanna Marçal FIAP IA,Mais um Goiano por aqui?,message,None,2225,2024-08-23 04:50:00
2225,23/08/2024,08:25,+55 11 96148-8581,Eu abri o app da fiap e tirei a foto do celula...,message,None,2226,2024-08-23 08:25:00
2226,23/08/2024,09:37,+55 11 98763-4164,Consegui tbm,message,None,2227,2024-08-23 09:37:00
2227,23/08/2024,09:52,+55 44 9760-4925,eu não consegui porque tinha os dados do cadas...,message,None,2228,2024-08-23 09:52:00
2228,23/08/2024,10:06,+55 11 96148-8581,<Mídia oculta>,message,None,2229,2024-08-23 10:06:00
...,...,...,...,...,...,...,...,...
2619,23/08/2024,23:26,+55 11 91319-9755,Hj tem show do cpm22 agora meia a noite e meia...,message,None,2620,2024-08-23 23:26:00
2620,23/08/2024,23:26,+55 11 91319-9755,Boaaa,message,None,2621,2024-08-23 23:26:00
2621,23/08/2024,23:27,+55 11 93800-6662,faltou ontem e hoje,message,None,2622,2024-08-23 23:27:00
2622,23/08/2024,23:27,+55 11 93800-6662,mas amanha faco isso,message,None,2623,2024-08-23 23:27:00


In [11]:
from pprint import pprint

# Full chat string
# Every new message starts with a new line followed by NM:
full_chat_str = " ".join(messages_df["Message"].values)

full_chat_str[:1000]

'Mais um Goiano por aqui? Eu abri o app da fiap e tirei a foto do celular com a carteirinha =) Consegui tbm eu não consegui porque tinha os dados do cadastro incompleto, depois eu tento denovo <Mídia oculta> <Mídia oculta> <Mídia oculta> <Mídia oculta> Opa, não sabia que tinha desconto pra notion também Agora gostei Já uso a anos, uma das melhores ferramentas para se rebelar em equipe. Pessoal quem não preencheu esse forms, preenche por favor; https://forms.gle/FgrU5R4fd71tC3Vg9 tem 52 resposta somente Exatamente! Nasci em MG, mas moro em Goiânia/GO quem já tem grupo formado so me fala no privado que ai tiro o nome do grupo todo, tendo em vista que se todos tiver de acordo são no total 20 grupos de 5 pessoas. dai a ideia era colocar quem sabe o intermediario de Python e avançado como lider do grupo sabe. só para separarmos rapido. O que acham? Preenchido,  tô meio por fora,  mas preenchi lá 😅 ORACLE UNIVERSITY\nINSCREVA-SE ATÉ O DIA 26/08, ÀS 23:59HS\nOlá, turma 1TIAOR-2024.\n\nCompart

In [12]:
from pprint import pprint

SUMMARY_PROMPT = f"""Por favor, resuma o seguinte bate-papo de grupo do WhatsApp com base nos tópicos discutidos. Para cada tópico, inclua seu título e um resumo em tópicos. Os tópicos devem incluir informações detalhadas. Se o tópico incluir recomendações sobre empresas ou serviços específicos, por favor, inclua-as no resumo. Inclua também os links que foram compartilhados."""
NEWSLETTER_PROMPT = f"""Por favor, forneça um parágrafo para abrir uma newsletter cobrindo os seguintes tópicos:"""

pprint(SUMMARY_PROMPT)

('Por favor, resuma o seguinte bate-papo de grupo do WhatsApp com base nos '
 'tópicos discutidos. Para cada tópico, inclua seu título e um resumo em '
 'tópicos. Os tópicos devem incluir informações detalhadas. Se o tópico '
 'incluir recomendações sobre empresas ou serviços específicos, por favor, '
 'inclua-as no resumo. Inclua também os links que foram compartilhados.')


In [13]:
def print_model(model):
    print("Uging model: ")
    print(model.dict())
    print('\n\n')

In [14]:
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# llm = ChatOpenAI(model="gpt-4o")
llm = ChatOpenAI(model="gpt-4o-mini")
print_model(llm)

code_prompt = PromptTemplate(
    template="{instruction}\nAs mensagens: ```\{messages}```",
    input_variables=["instruction", "messages"],
)

code_chain = code_prompt | llm

langchain.debug = True


Uging model: 
{'model_name': 'gpt-4o-mini', 'model': 'gpt-4o-mini', 'stream': False, 'n': 1, 'temperature': 0.7, '_type': 'openai-chat'}





In [15]:

# results = code_chain.invoke({
#     "instruction": SUMMARY_PROMPT,
#     "messages": full_chat_str,
# })
# results

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "instruction": "Por favor, resuma o seguinte bate-papo de grupo do WhatsApp com base nos tópicos discutidos. Para cada tópico, inclua seu título e um resumo em tópicos. Os tópicos devem incluir informações detalhadas. Se o tópico incluir recomendações sobre empresas ou serviços específicos, por favor, inclua-as no resumo. Inclua também os links que foram compartilhados.",
  "messages": "Mais um Goiano por aqui? Eu abri o app da fiap e tirei a foto do celular com a carteirinha =) Consegui tbm eu não consegui porque tinha os dados do cadastro incompleto, depois eu tento denovo <Mídia oculta> <Mídia oculta> <Mídia oculta> <Mídia oculta> Opa, não sabia que tinha desconto pra notion também Agora gostei Já uso a anos, uma das melhores ferramentas para se rebelar em equipe. Pessoal quem não preencheu esse forms, preenche por favor; https://forms.gle/FgrU5R4fd71tC3Vg9 tem 52 resposta somente Exatamente! Nasci em MG, mas 

AIMessage(content="**Resumo do Bate-Papo do Grupo do WhatsApp**\n\n---\n\n### 1. Grupos de Estudo e Formação\n- **Formação de Grupos:** A maioria dos alunos está se organizando em grupos para atividades, com o uso de um formulário compartilhado para facilitar a formação. O link para o formulário é: [Formulário de Formação de Grupos](https://forms.gle/FgrU5R4fd71tC3Vg9).\n- **Divisão de Grupos:** Sugestão para formar grupos de 5 pessoas com diferentes níveis de conhecimento em Python. A ideia é que alunos com conhecimento intermediário ou avançado atuem como líderes.\n- **Necessidade de Novos Integrantes:** Muitos alunos estão à procura de vagas em grupos ou formando novos grupos e pedindo para se juntar.\n\n### 2. Oportunidades de Estágio e Trainee\n- **Vagas Compartilhadas:** Vários links de oportunidades de trainee foram compartilhados:\n  - Trainee Itaú: [Link](https://lnkd.in/dTexsixN)\n  - Trainee Ambev: [Link](https://lnkd.in/dWqeC3JM)\n  - Trainee Nestlé: [Link](https://lnkd.in/

In [22]:
pprint(results.content)

# copy to clipboard:
import pyperclip
pyperclip.copy(results.content)

('**Resumo do Bate-Papo do Grupo do WhatsApp**\n'
 '\n'
 '---\n'
 '\n'
 '### 1. Grupos de Estudo e Formação\n'
 '- **Formação de Grupos:** A maioria dos alunos está se organizando em grupos '
 'para atividades, com o uso de um formulário compartilhado para facilitar a '
 'formação. O link para o formulário é: [Formulário de Formação de '
 'Grupos](https://forms.gle/FgrU5R4fd71tC3Vg9).\n'
 '- **Divisão de Grupos:** Sugestão para formar grupos de 5 pessoas com '
 'diferentes níveis de conhecimento em Python. A ideia é que alunos com '
 'conhecimento intermediário ou avançado atuem como líderes.\n'
 '- **Necessidade de Novos Integrantes:** Muitos alunos estão à procura de '
 'vagas em grupos ou formando novos grupos e pedindo para se juntar.\n'
 '\n'
 '### 2. Oportunidades de Estágio e Trainee\n'
 '- **Vagas Compartilhadas:** Vários links de oportunidades de trainee foram '
 'compartilhados:\n'
 '  - Trainee Itaú: [Link](https://lnkd.in/dTexsixN)\n'
 '  - Trainee Ambev: [Link](https://lnk

In [20]:
results.usage_metadata['total_tokens']

7112

In [23]:
messages_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397 entries, 2224 to 2623
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         397 non-null    object
 1   Time         397 non-null    object
 2   Id           397 non-null    object
 3   Message      397 non-null    object
 4   Type         397 non-null    object
 5   Action_type  0 non-null      object
 6   Order        397 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 24.8+ KB
